In [1]:
import json
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
#from Preprocessing import Preprocessor
from constants import RUSSIAN_TIME_ZONES, rp5_columns
from interpolation import interpolate_mmx, create_patterns
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_mmx_to_mmcc_forecast, convert_raw_to_mmx
from loaders import load_rp5_stations, load_mm94_stations
from geographical import find_nearest_wmo_station
from Preprocessing import vincenty_dist

mypath = '/mnt/HARD/MinMax94/data/CSV'

## Convert RP5 to Mmx

* Choose one mm94 station and finding the closeat wmo station

In [8]:
## find nearest wmo station
mm94_station_id = 20833

wmo_station_id = find_nearest_wmo_station(mm94_station_id)
print(wmo_station_id.min())

MIN:  11.426329335
23412


* Load data from the nearest wmo station

In [3]:
rp5 = load_rp5_stations(wmo_station_id)

* Convert from RP5 to Mmx format

In [4]:
mmx_forecast = convert_rp5_to_mmx(rp5)
mmx_forecast_interpolated = interpolate_mmx(mmx_forecast, 20)
mmx_forecast_interpolated

,date_time_utc,data_pressure,data_visibility,data_precip_count,station_id,data_dew_point,data_precip_code,date_time,data_cloudiness,data_wind_speedmax,data_p_weather,data_t_air,data_wind_dir,data_dampness,data_wind_velocity
0,2013-01-11 12:00:00,760.800000,20000.000000,NaN,35700,-10.700000,20.0,2013-01-11 17:00:00,75.000000,NaN,0.0,-8.800000,112.5,86.000000,3.000000
1,2013-01-11 12:20:00,760.911111,20000.000000,NaN,35700,-10.766667,20.0,2013-01-11 17:20:00,77.777778,NaN,0.0,-8.933333,112.5,86.444444,3.000000
2,2013-01-11 12:40:00,761.022222,20000.000000,NaN,35700,-10.833333,20.0,2013-01-11 17:40:00,80.555556,NaN,0.0,-9.066667,112.5,86.888889,3.000000
3,2013-01-11 13:00:00,761.133333,20000.000000,NaN,35700,-10.900000,20.0,2013-01-11 18:00:00,83.333333,NaN,0.0,-9.200000,112.5,87.333333,3.000000
4,2013-01-11 13:20:00,761.244444,20000.000000,NaN,35700,-10.966667,20.0,2013-01-11 18:20:00,86.111111,NaN,0.0,-9.333333,112.5,87.777778,3.000000
5,2013-01-11 13:40:00,761.355556,20000.000000,NaN,35700,-11.033333,20.0,2013-01-11 18:40:00,88.888889,NaN,0.0,-9.466667,112.5,88.222222,3.000000
6,2013-01-11 14:00:00,761.466667,20000.000000,NaN,35700,-11.100000,20.0,2013-01-11 19:00:00,91.666667,NaN,0.0,-9.600000,112.5,88.666667,3.000000
7,2013-01-11 14:20:00,761.577778,20000.000000,NaN,35700,-11.166667,20.0,2013-01-11 19:20:00,94.444444,NaN,0.0,-9.733333,112.5,89.111111,3.000000
8,2013-01-11 14:40:00,761.688889,20000.000000,NaN,35700,-11.233333,20.0,2013-01-11 19:40:00,97.222222,NaN,0.0,-9.866667,112.5,89.555556,3.000000
9,2013-01-11 15:00:00,761.800000,20000.000000,0.083333,35700,-11.300000,20.0,2013-01-11 20:00:00,100.000000,NaN,0.0,-10.000000,112.5,90.000000,3.000000


* Convert from Mmx to MmccForecast format

In [5]:
mmcc_forecast = convert_mmx_to_mmcc_forecast(mmx_forecast_interpolated)

## Convert Raw to Mmx

* Load Raw mm94 data

In [6]:
%%time
raw = load_mm94_stations(mm94_station_id)
raw.head()

CPU times: user 1.84 s, sys: 120 ms, total: 1.96 s
Wall time: 1.96 s


* Convert Raw to Mmx

In [7]:
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis, 20)
mmx_rwis_interpolated = create_patterns(mmx_rwis_interpolated)

Column id_p_weather can not be calculated
Column data_visibility can not be calculated
Column id_visibility can not be calculated
Column data_p_weather can not be calculated


In [9]:
mmcc_rwis = convert_mmx_to_mmcc_forecast(mmx_rwis_interpolated)
mmcc_rwis

Column visibility can not be calculated
Column p_weather can not be calculated


,date_time_metro,wind_direction,wind_speed,precipitation_type,station_id,t_air,humidity,pressure,t_dew_point,precipitation_intensity,visibility,date_time_utc,p_weather,cloudiness
19,2008-03-23 21:00 UTC,9999.000000,9999.0,9999.0,1821,-2.733333,97.975000,9999.000000,-2.925000,9999.0,9999,2008-03-23 21:00:00,9999,9999.0
20,2008-03-23 21:20 UTC,9999.000000,9999.0,9999.0,1821,-2.666667,98.050000,9999.000000,-2.850000,9999.0,9999,2008-03-23 21:20:00,9999,9999.0
21,2008-03-23 21:40 UTC,9999.000000,9999.0,9999.0,1821,-2.600000,98.125000,9999.000000,-2.775000,9999.0,9999,2008-03-23 21:40:00,9999,9999.0
22,2008-03-23 22:00 UTC,9999.000000,9999.0,9999.0,1821,-2.533333,98.200000,9999.000000,-2.700000,9999.0,9999,2008-03-23 22:00:00,9999,9999.0
23,2008-03-23 22:20 UTC,9999.000000,9999.0,9999.0,1821,-2.466667,98.275000,9999.000000,-2.625000,9999.0,9999,2008-03-23 22:20:00,9999,9999.0
24,2008-03-23 22:40 UTC,9999.000000,9999.0,9999.0,1821,-2.400000,98.350000,9999.000000,-2.550000,9999.0,9999,2008-03-23 22:40:00,9999,9999.0
25,2008-03-23 23:00 UTC,9999.000000,9999.0,9999.0,1821,-2.333333,98.425000,9999.000000,-2.475000,9999.0,9999,2008-03-23 23:00:00,9999,9999.0
26,2008-03-23 23:20 UTC,9999.000000,9999.0,9999.0,1821,-2.266667,98.500000,9999.000000,-2.400000,9999.0,9999,2008-03-23 23:20:00,9999,9999.0
27,2008-03-23 23:40 UTC,9999.000000,9999.0,9999.0,1821,-2.200000,98.575000,9999.000000,-2.325000,9999.0,9999,2008-03-23 23:40:00,9999,9999.0
28,2008-03-24 00:40 UTC,9999.000000,9999.0,9999.0,1821,-2.050000,98.575000,9999.000000,-2.175000,9999.0,9999,2008-03-24 00:40:00,9999,9999.0
